In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
%matplotlib inline
from sklearn.utils import shuffle
import matplotlib.image as mpimg
import tensorflow as tf
from skimage.transform import resize
from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5501434706769672290
]


In [4]:
fpP = "../data/Parasitized/"
fpU = "../data/Uninfected/"

In [5]:
dataP = []
num_imgs = 1000
for filename in os.listdir(fpP):
    while num_imgs > 0:
        if '.png' in filename:
            num_imgs -= 1
            dataP.append(resize(mpimg.imread(fpP+filename), (199,199,3)))

In [ ]:
dataU = []
num_imgs = 1000
for filename in os.listdir(fpU):
    if '.png' in filename:
        num_imgs-=1
        dataU.append(resize(mpimg.imread(fpU+filename), (199, 199, 3)))

In [ ]:
dataP = np.array(dataP)
dataU = np.array(dataU)

In [ ]:
plt.imshow(dataP[0])


In [ ]:
plt.imshow(dataU[0])

In [ ]:
# Merge data
p = np.ones(dataP.shape[0])
u = np.zeros(dataU.shape[0])
X = np.concatenate((dataP, dataU))
y = np.concatenate((p, u))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Normalize image vectors
X_train = X_train/255.
X_test = X_test/255.

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("y_test shape: " + str(y_test.shape))

In [ ]:
img_height,img_width = 199,199 
num_classes = 1
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = applications.resnet50.ResNet50(weights= None,
                                            include_top=False,
                                            input_shape= (img_height,img_width,3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
from keras.optimizers import SGD, Adam
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
len(y_train)

In [ ]:
model.fit(X_train, y_train, epochs = 100, batch_size = 64)